In [1]:
import os
import cv2
import sqlite3
import numpy as np

# Connect to SQLite database
conn = sqlite3.connect('face_recognition.db')
cursor = conn.cursor()

# Create table to store face images
cursor.execute('''
CREATE TABLE IF NOT EXISTS faces (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    image BLOB
)
''')

# Function to convert image to binary
def convert_to_binary(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (64, 64))  # Ensure consistent image size
    _, buffer = cv2.imencode('.jpg', image)
    return buffer.tobytes()

# Path to the dataset directory
dataset_path = 'dataset_img'

# Iterate through each person (folder)
for person_name in os.listdir(dataset_path):
    person_folder = os.path.join(dataset_path, person_name)
    
    # Check if it's a directory
    if os.path.isdir(person_folder):
        for image_name in os.listdir(person_folder):
            image_path = os.path.join(person_folder, image_name)
            
            # Convert image to binary
            image_blob = convert_to_binary(image_path)
            
            # Insert image into the database
            cursor.execute('INSERT INTO faces (name, image) VALUES (?, ?)', (person_name, image_blob))
            conn.commit()

conn.close()
print("Images and labels have been inserted into the database.")

Images and labels have been inserted into the database.


In [22]:
import numpy as np
import cv2
import sqlite3

def fetch_images_from_db():
    conn = sqlite3.connect('face_recognition.db')
    cursor = conn.cursor()

    cursor.execute('SELECT name, image FROM faces')
    rows = cursor.fetchall()
    conn.close()

    names = []
    images = []

    for row in rows:
        name, image_blob = row
        image_array = np.frombuffer(image_blob, np.uint8)
        image = cv2.imdecode(image_array, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (200, 200))  # Ensure consistent size
        names.append(name)
        images.append(image)

    return names, images

def preprocess_images(images):
    processed_images = []
    for image in images:
        processed_images.append(image.flatten())
    return np.array(processed_images)

In [18]:
from sklearn.decomposition import PCA

names, images = fetch_images_from_db()
X = preprocess_images(images)

n_components = min(X.shape[0], X.shape[1], 50)  # Limit to 50 components or the minimum of samples/features
pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X)


In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_pca, names, test_size=0.2, random_state=42)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Evaluate the model
accuracy = rf.score(X_test, y_test)
print("Model accuracy:", accuracy)


Model accuracy: 0.2903225806451613


In [20]:
# Initialize webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_gray_resized = cv2.resize(roi_gray, (200, 200))  # Ensure consistent size
        flattened_face = roi_gray_resized.flatten()
        pca_face = pca.transform(flattened_face.reshape(1, -1))

        # Predict the person
        probabilities = rf.predict_proba(pca_face)[0]
        max_prob_index = np.argmax(probabilities)
        confidence = probabilities[max_prob_index]
        if confidence > 0.5:  # Adjust the threshold as needed
            name = rf.classes_[max_prob_index]
        else:
            name = "Unknown"

        # Draw a rectangle around the face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Put name outside the rectangle
        cv2.putText(frame, name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)

    # Display the resulting frame
    cv2.imshow('Face Recognition', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
